In [6]:
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm

In [7]:
# 0. 파일 경로 및 설정
topic_file = "df_final_hawk_dove_topic_prob.csv"  # 토픽 모델링 결과 파일
macro_file = "macro_data.csv"             # 경제 지표 파일

print(">>> 1. 토픽 모델링 결과 로드 중...")
df_topic = pd.read_csv(topic_file)

>>> 1. 토픽 모델링 결과 로드 중...


In [8]:
# 1. 전처리 - 날짜 추출 & fomc 필터링
# (1) 날짜 추출 함수
def extract_date_from_id(doc_id):
    match = re.search(r"(\d{8})", str(doc_id))
    if match:
        return match.group(1)
    return None

df_topic['DATE'] = df_topic['doc_id'].apply(extract_date_from_id)
df_topic['DATE'] = pd.to_datetime(df_topic['DATE'], format='%Y%m%d', errors='coerce')

print("\n>>> 2. FOMC 의사록 필터링 중...")

def is_minutes(doc_id):
    return 'FOMC' in str(doc_id).upper()

# 필터링 및 복사
df_target = df_topic[df_topic['doc_id'].apply(is_minutes)].copy()

# (3) 핵심 점수 칼럼 이름 변경
# doc_hawk_dove_score_prob -> Topic_Tone
df_target = df_target.rename(columns={'doc_hawk_dove_score_prob': 'Topic_Tone'})

# 날짜순 정렬
df_target = df_target.sort_values('DATE')

print(f"   - 전체 데이터 개수: {len(df_topic)}개")
print(f"   - 필터링된 FOMC 의사록 개수: {len(df_target)}개")
print(f"   - 데이터 기간: {df_target['DATE'].min().date()} ~ {df_target['DATE'].max().date()}")


>>> 2. FOMC 의사록 필터링 중...
   - 전체 데이터 개수: 462053개
   - 필터링된 FOMC 의사록 개수: 298062개
   - 데이터 기간: 1993-02-03 ~ 2025-09-17


In [10]:
# 2. 월별 집계
df_target['YearMonth'] = df_target['DATE'].dt.to_period('M').dt.to_timestamp()

# Topic_Tone 평균 계산
monthly_tone = df_target.groupby('YearMonth')[['Topic_Tone']].mean().reset_index()

In [11]:
# 3. 경제 데이터 병합 및 테일러 준칙 변수 생성
print("\n>>> 3. 거시경제 데이터 병합 및 변수 생성...")

df_macro = pd.read_csv(macro_file)
df_macro['DATE'] = pd.to_datetime(df_macro['DATE'])
df_macro['YearMonth'] = df_macro['DATE'].dt.to_period('M').dt.to_timestamp()

# 병합 (Inner Join)
df = pd.merge(df_macro, monthly_tone, on='YearMonth', how='inner').sort_values('YearMonth')

# 테일러 준칙용 변수 생성 (차분 및 시차)
# 1. 종속변수: 금리 변화 (이번달 - 지난달)
df['Delta_Base_Rate'] = df['Base_Rate'].diff()

# 2. 설명변수 1: 금리 평활화(Smoothing)를 위한 지난달 금리 변화
df['Lag_Delta_Base'] = df['Delta_Base_Rate'].shift(1)

# 3. 설명변수 2: 인플레이션 변화 (Gap 대신 변화량을 쓰는 경우)
df['Delta_Inflation'] = df['CPI'].diff()

# 4. 설명변수 3: 경기(산생지수) 변화율
df['Delta_Output'] = df['IP'].pct_change() * 100

# 결측치 제거 (diff, shift로 생긴 NaN 제거)
df = df.dropna()

print(f"   - 최종 분석에 사용되는 데이터 행 개수: {len(df)}개")


>>> 3. 거시경제 데이터 병합 및 변수 생성...
   - 최종 분석에 사용되는 데이터 행 개수: 259개


In [14]:
# 4. 회귀분석
print("="*60)
print("             [토픽 모델링 기반 테일러 준칙 분석]             ")
print("="*60)

# 분석할 어조 변수
target_tone = 'Topic_Tone'

# X: 설명변수 (상수항 + 기존 경제변수 + 토픽 어조)
X = df[['Lag_Delta_Base', 'Delta_Inflation', 'Delta_Output', target_tone]]
X = sm.add_constant(X)
y = df['Delta_Base_Rate']

# 모델 학습
model = sm.OLS(y, X).fit()

# 결과 출력
print(f"\n분석 대상 변수: {target_tone} (doc_hawk_dove_score_prob)")
print("-" * 40)
print(f"   ▶ R-squared (설명력) : {model.rsquared:.4f}")
print(f"   ▶ Coef (어조 계수)   : {model.params[target_tone]:.4f}")
print(f"   ▶ P-value (유의확률) : {model.pvalues[target_tone]:.4f}")

if model.pvalues[target_tone] < 0.05:
    print("\n   🎉 축하합니다! 토픽 모델링 점수가 통계적으로 유의미합니다. (P < 0.05)")
    if model.params[target_tone] > 0:
        print("      해석: 어조가 매파적일수록(점수 높음) -> 금리가 인상되는 경향이 확인되었습니다.")
    else:
        print("      해석: 계수가 음수입니다. 점수 방향성(-1/1)을 다시 확인해볼 필요가 있습니다.")
else:
    print("\n   ⚠️ 아쉽게도 통계적으로 유의하지 않습니다. (P > 0.05)")

print("\n" + "="*60)
# print(model.summary()) # 자세한 표가 보고 싶으면 주석 해제

             [토픽 모델링 기반 테일러 준칙 분석]             

분석 대상 변수: Topic_Tone (doc_hawk_dove_score_prob)
----------------------------------------
   ▶ R-squared (설명력) : 0.3746
   ▶ Coef (어조 계수)   : 1.7680
   ▶ P-value (유의확률) : 0.5304

   ⚠️ 아쉽게도 통계적으로 유의하지 않습니다. (P > 0.05)

